# Actividad Práctica: Analsisis de Señales de Electrocardiograma (ECG)


### Objetivo General
El propósito de este curso es introducir al estudiante en técnicas de análisis exploratorio y visualización interactiva de datos fisiológicos, enfocándose en curvas electrocardiográficas (ECG). Se abordarán técnicas esenciales para la limpieza, manipulación y visualización de datos con el fin de obtener métricas clínicas útiles como frecuencia cardíaca, variabilidad entre latidos (RR) y desarrollar interfaces que faciliten su uso clínico.

### Al finalizar este curso, el estudiante estará en capacidad de:

- Leer y limpiar señales electrocardiográficas (ECG), identificando artefactos y eliminando segmentos no informativos.
- Identificar metadatos relevantes de registros de señales: frecuencia de muestreo, cantidad de canales y unidades.
- Calcular métricas clave como frecuencia cardíaca y duración de intervalos RR.
- Extraer y segmentar latidos individuales o fragmentos específicos del ECG.
- Visualizar datos en forma gráfica, incluyendo:
  - Promedios de latidos cardíacos,
  - Curvas de error/desviación,
  - Cambios a lo largo del tiempo.
- Implementar dashboards interactivos para monitoreo en tiempo real con métricas clave.
- Integrar sliders y controles gráficos para explorar dinámicamente la señal ECG en diferentes ventanas de tiempo.
- Diseñar sistemas básicos de alerta para entornos clínicos que utilicen la información visualizada en tiempo real.





## 1. Base de datos de ECG

Al igual que en curso 1 de este programa, se utilizará la base de datos de ECG de PhysioNet, que contiene registros de ECG de pacientes con arritmias y otros trastornos cardíacos.


#### MIT-BIH Arrhythmia Database


![](/Users/carlos/Documents/github/Diplomado_iHealth/img/mitbih.png)


Link a la base de datos: https://www.physionet.org/content/mitdb/1.0.0/
Esta cuenta con 48 registros de ECG de 30 minutos de duración, con una frecuencia de muestreo de **360 Hz** y **2 canales**. Los registros están muestreados a 11 bits con un rango de 10 mV y un filtro pasa banda de 0.1-100 Hz. Los registros están etiquetados por dos cardiólogos expertos.

Cada grabación cuenta con 4 archivos:
- `.xms`: configuración para visualizar el registro en el navegador.
- `.atr`: contiene las anotaciones (en terminos de muestras) de las ondas R de cada latido.
- `.hea`: contiene información sobre el registro, como la frecuencia de muestreo, el número de canales, la duración del registro, etc.
- `.dat`: contiene las señales de ECG.

> **NOTA**
> Los formatos de datos de ECG y otras señales fisiologicas son variados. Ejemplos de formatos de datos de ECG son: `.dat`, `.csv`, `.txt`, `.mat`, `.edf`, etc. Es importante conocer el formato de los datos para poder leerlos y trabajar con ellos con las herramientas adecuadas.

En este ejercicio solo se utilizara un registro de ECG para simplificar el análisis.

## Librerías necesarias

En esta actividad se utilizarán las siguientes librerías de Python:

- `wfdb`, para la lectura y manejo de señales ECG provenientes de bases de datos fisiológicas como MIT-BIH.
- `numpy`, para la manipulación y análisis numérico de los datos.
- `matplotlib`, para la visualización gráfica de las señales.
- `os`, para operaciones relacionadas con el sistema de archivos y la gestión de directorios.
- `scipy`, para el procesamiento de señales.



In [ ]:
# Instalando Librerias
!pip install wfdb


In [ ]:
# cargando librerias
import wfdb
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import find_peaks

## 2. Lectura de datos

#### **Ejercicio 1** 

Utilizando la librería `wfdb`, lee el registro de ECG `105` de la base de datos MIT-BIH. Identifica que la frequencia de muestro es de 360 Hz, que el registro tiene 2 canales y que la duración del registro es de 30 minutos. Imprime esta información en pantalla. Se deben crear las variables `ecg`, `fs`, `duration`, `n_channels` para almacenar la señal de ECG, la frecuencia de muestreo, duración y el número de canales, respectivamente. HINT: utiliza la función `wfdb.dl_database` para descargar el registro y `wfdb.rdsamp` para leerlo.



In [ ]:
# <CODE>


#### **Ejercicio 2** 

Utilizando subplots crea un plot mostrando los primeros 60 segundos de cada canal del registro de ECG. Utiliza la función `plt.subplots` para crear los subplots y `plt.plot` para graficar las señales. Asegúrate de etiquetar los ejes y agregar un título a cada subplot indicando el canal correspondiente.



In [ ]:
# <CODE>


#### **Ejercicio 3** 

En la grafica anterior se pueden observar algunos valores atípicos en la señal de ECG. Por ejemplo en el canal 2, se observa que aproximadamente en los segundos 15, 21 y 35 hay valores que se alejan significativamente del rango normal de la señal. Identifica los índices de estos valores atípicos en el canal 2 y almacénalos en una variable llamada `outliers`. Elimina estos valores de la señal de ECG y almacena la señal limpia en una nueva variable llamada `ecg_clean`. Puedes considerar que todo lo que supere un treshold de 0.4mV en el canal 2 es un valor atípico


In [ ]:
# Crear máscara con np.where
outliers = (ecg[:, 1] > 0.4)

ecg_clean = ecg.copy()
ecg_clean[outliers, :] = 0  # Establecer a cero los valores donde la máscara es True

# Graficar la señal ECG después de aplicar la máscara
fig, ax = plt.subplots(2,1, figsize=(12, 10))
ax[0].plot(tiempo, ecg_clean[:, 0], color='blue')
ax[1].plot(tiempo, ecg_clean[:, 1], color='blue')
ax[0].set_xlim(0, 60)
ax[1].set_xlim(0, 60)
ax[0].set_title('Señal ECG - Canal 1 (Después de aplicar máscara)')
ax[1].set_title('Señal ECG - Canal 2 (Después de aplicar máscara)')
ax[0].set_xlabel('Tiempo (s)')
ax[1].set_xlabel('Tiempo (s)')
ax[0].set_ylabel('Amplitud')
ax[1].set_ylabel('Amplitud')
ax[0].grid()
ax[1].grid()
plt.show()





## 3. Manipulación de la señal
#### **Ejercicio 4** 

Para entender el comportamiento de la señal de ECG, es importante calcular la frecuencia cardíaca. Para esto es necesario obtener los tiempos de los segmentos R de la señal de ECG. Anteriormente desarrollamos una función para detectar los picos de la señal, sin embargo librerias como `scipy` cuentan con funciones que facilitan esta tarea. Utilizando la función `find_peaks` de `scipy.signal`, identifica los picos de la señal de ECG original y almacena los índices de estos picos en una variable llamada `r_peaks`. Utiliza el canal 1 para la detección de picos y genera un plot destacando los peaks. Responde las siguientes preguntas:
* ¿Que inputs son necesarios para la función `find_peaks`?
* ¿Que parámetros son necesarios para la detección de picos en la señal de ECG?
* ¿Entre que rango de amplitudes se encuentran los picos de la señal de ECG?

In [ ]:
# <CODE>


#### **Ejercicio 5** 

Deacuerdo al ejercicio anterior fue posible identificar la presencia de valores R-R atipicos en la señal de ECG. Utilizando la variable `r_peaks` y la función `np.diff`, calcula los intervalos R-R de la señal de ECG. Almacena estos intervalos en una variable llamada `rr_intervals`. Utiliza la frecuencia de muestreo para convertir los intervalos R-R a segundos y almacena el resultado en una nueva variable llamada `rr_intervals_sec`. Responde las siguientes preguntas:
* ¿Que unidades tiene la variable `rr_intervals_sec`?
* ¿Que rango de valores tiene la variable `rr_intervals_sec`?
* ¿Existen valores fuera del rango normal de la frecuencia cardíaca? (60-100 latidos por minuto)




In [ ]:
# <CODE>
